# Stadium Billiard Assignment — GIF Animations

This notebook generates and saves GIF animations for both the classical stadium billiard trajectory and the quantum wave packet evolution inside the stadium. The animations visually demonstrate chaotic dynamics and quantum evolution in the stadium geometry.

## 1. Generate and Save Stadium Billiard Trajectory GIF (Classical)

We animate the trajectory of a classical particle inside the stadium billiard using `matplotlib.animation.FuncAnimation`. The animation shows the particle bouncing off the boundaries, illustrating chaotic motion. The resulting GIF is saved for easy sharing and visualization.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Stadium parameters
a = 1.0   # half-width of rectangle
b = 1.0   # half-height of rectangle
rx = 2.0  # ellipse semi-axis in x (caps)
ry = b    # ellipse semi-axis in y

# Simulate trajectory (reuse function from previous notebook)
def ellipse_quadratic_coeffs(x0, y0, vx, vy, cx):
    dx0 = x0 - cx
    A = (vx*vx)/(rx*rx) + (vy*vy)/(ry*ry)
    B = 2*(dx0*vx)/(rx*rx) + 2*(y0*vy)/(ry*ry)
    C = (dx0*dx0)/(rx*rx) + (y0*y0)/(ry*ry) - 1.0
    return A, B, C

def solve_quadratic(A, B, C):
    if abs(A) < 1e-18:
        if abs(B) < 1e-18:
            return []
        t = -C / B
        return [t] if np.isfinite(t) else []
    disc = B*B - 4*A*C
    if disc < 0:
        return []
    sqrt_d = np.sqrt(disc)
    t1 = (-B - sqrt_d) / (2*A)
    t2 = (-B + sqrt_d) / (2*A)
    roots = []
    for t in (t1, t2):
        if np.isfinite(t):
            roots.append(t)
    roots = sorted(set(roots))
    return roots

def ellipse_normal_at(x, y, cx):
    nx = (x - cx)/(rx*rx)
    ny = y/(ry*ry)
    v = np.array([nx, ny])
    norm = np.linalg.norm(v)
    if norm == 0:
        return np.array([1.0, 0.0])
    return v / norm

def reflect(v, n):
    return v - 2*np.dot(v, n)*n

def kinetic_energy(vx, vy, mass=1.0):
    return 0.5 * mass * (vx**2 + vy**2)

def simulate_trajectory(x0, y0, vx0, vy0, dt=0.01, steps=2000, max_bounces=None, eps_nudge=1e-9):
    x, y = float(x0), float(y0)
    vx, vy = float(vx0), float(vy0)
    X = [x]
    Y = [y]
    bounces = []
    bounce_count = 0
    for step in range(steps):
        remaining = dt
        while remaining > 1e-16:
            candidates = []
            if abs(vy) > 1e-16:
                for wall_y in (b, -b):
                    t = (wall_y - y)/vy
                    if t > 1e-15 and t <= remaining + 1e-15:
                        x_at = x + vx*t
                        if abs(x_at) <= a + 1e-12:
                            ny = 1.0 if wall_y > 0 else -1.0
                            if np.dot(np.array([vx, vy]), np.array([0.0, ny])) > 1e-12:
                                candidates.append((t, 1, 'hwall', wall_y, None))
            for sx in (1.0, -1.0):
                cx = sx * a
                A, B, C = ellipse_quadratic_coeffs(x, y, vx, vy, cx)
                roots = solve_quadratic(A, B, C)
                for t in roots:
                    if t > 1e-15 and t <= remaining + 1e-15:
                        x_at = x + vx*t
                        y_at = y + vy*t
                        if sx > 0 and x_at >= a - 1e-12:
                            n_ = ellipse_normal_at(x_at, y_at, cx)
                            if np.dot(np.array([vx, vy]), n_) > 1e-12:
                                candidates.append((t, 0, 'ellipse', cx, sx))
                        if sx < 0 and x_at <= -a + 1e-12:
                            n_ = ellipse_normal_at(x_at, y_at, cx)
                            if np.dot(np.array([vx, vy]), n_) > 1e-12:
                                candidates.append((t, 0, 'ellipse', cx, sx))
            if not candidates:
                x += vx * remaining
                y += vy * remaining
                remaining = 0.0
                break
            candidates.sort(key=lambda c: (c[0], c[1]))
            t_coll, _, kind, param, extra = candidates[0]
            x = x + vx * t_coll
            y = y + vy * t_coll
            if kind == 'hwall':
                ny = 1.0 if param > 0 else -1.0
                n = np.array([0.0, ny])
                v_ref = reflect(np.array([vx, vy]), n)
            else:
                cx = param
                n = ellipse_normal_at(x, y, cx)
                v_ref = reflect(np.array([vx, vy]), n)
            bounce_count += 1
            bounces.append((len(X), x, y, kind))
            v_ref = np.array(v_ref, dtype=float)
            speed = np.hypot(v_ref[0], v_ref[1])
            if speed > 0:
                v_ref = v_ref * (np.hypot(vx, vy) / speed)
            vx, vy = float(v_ref[0]), float(v_ref[1])
            vnorm = np.hypot(vx, vy)
            if vnorm > 0:
                x += (vx / vnorm) * eps_nudge
                y += (vy / vnorm) * eps_nudge
            remaining -= t_coll
            if max_bounces is not None and bounce_count >= max_bounces:
                remaining = 0.0
                break
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y), vx, vy, bounces

# Generate trajectory
X, Y, vx_final, vy_final, bounces = simulate_trajectory(0.0, 0.0, 1, 1, dt=0.01, steps=2000)

# Prepare stadium boundary for plotting
Xt = np.linspace(-a, a, 200)
Yt = np.full_like(Xt, b)
Xb_right = a + rx * np.cos(np.linspace(np.pi/2, -np.pi/2, 200))
Yb_right = ry * np.sin(np.linspace(np.pi/2, -np.pi/2, 200))
Xb_full = np.concatenate([Xt, Xb_right, Xt[::-1], -Xb_right[::-1]])
Yb_full = np.concatenate([Yt, Yb_right, -Yt, Yb_right[::-1]])

# Animation setup
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(Xb_full, Yb_full, 'k-', lw=1.2)
ax.set_aspect('equal')
ax.set_xlim(Xb_full.min()-0.2, Xb_full.max()+0.2)
ax.set_ylim(Yb_full.min()-0.2, Yb_full.max()+0.2)
ax.grid(True)
traj_line, = ax.plot([], [], '-b', lw=1.2)
particle, = ax.plot([], [], 'ro', markersize=8)
start_point, = ax.plot([X[0]], [Y[0]], 'go', markersize=8, label='start')
ax.legend()

def init():
    traj_line.set_data([], [])
    particle.set_data([], [])
    return traj_line, particle

def animate(i):
    traj_line.set_data(X[:i+1], Y[:i+1])
    particle.set_data(X[i], Y[i])
    return traj_line, particle

frames = len(X)
ani = animation.FuncAnimation(fig, animate, frames=frames, init_func=init, blit=True, interval=10)

# Save as GIF
ani.save('stadium_billiard_trajectory.gif', writer='pillow', fps=60)
plt.close(fig)
print("Classical stadium billiard trajectory GIF saved as 'stadium_billiard_trajectory.gif'.")

Classical stadium billiard trajectory GIF saved as 'stadium_billiard_trajectory.gif'.


## 2. Generate and Save Quantum Wave Packet Evolution GIF

We animate the evolution of the quantum probability density inside the stadium billiard using `matplotlib.animation.FuncAnimation`. The animation shows the spreading and reflection of the wave packet, illustrating quantum dynamics in a chaotic geometry. The GIF is saved for visualization.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Quantum parameters and grid (reuse from previous notebook)
hbar = 1.0
m = 1.0
Nx, Ny = 128, 128  # Use smaller grid for faster animation
dx = dy = 0.05
dt = 0.001
T_steps = 200  # Fewer steps for GIF
sigma = 0.2
x0, y0 = 0.0, 0.0
px, py = 2.0, 0

x = np.linspace(-a - rx, a + rx, Nx) * 1.2
y = np.linspace(-b - ry, b + ry, Ny) * 1.2
X, Y = np.meshgrid(x, y)

def stadium_mask(X, Y, a, b, rx, ry):
    rect = (np.abs(X) <= a) & (np.abs(Y) <= b)
    left_ellipse = ((X + a)**2 / rx**2 + Y**2 / ry**2 <= 1) & (X <= -a)
    right_ellipse = ((X - a)**2 / rx**2 + Y**2 / ry**2 <= 1) & (X >= a)
    return rect | left_ellipse | right_ellipse

mask = stadium_mask(X, Y, a, b, rx, ry)
V = 1e6 * (1 - mask)

# Momentum grid
kx = 2 * np.pi * np.fft.fftfreq(Nx, dx)
ky = 2 * np.pi * np.fft.fftfreq(Ny, dy)
KX, KY = np.meshgrid(kx, ky)
k2 = (KX**2 + KY**2) * hbar**2 / (2 * m)

# Initial Gaussian wave packet
psi = np.sqrt(1 / (np.pi * sigma**2)) * np.exp(-((X - x0)**2 + (Y - y0)**2) / (2 * sigma**2) + 1j * (px * X + py * Y) / hbar)
psi *= mask
norm = np.sum(np.abs(psi)**2) * dx * dy
psi /= np.sqrt(norm)

def evolve_psi(psi, k2, dt, V):
    V_finite = np.where(np.isinf(V), 0, V)
    exp_factor = np.exp(-1j * V_finite * dt / (2 * hbar))
    psi = psi * exp_factor
    psi = np.where(np.isinf(V), 0, psi)
    psi_k = np.fft.fft2(psi)
    psi_k *= np.exp(-1j * k2 * dt / hbar)
    psi = np.fft.ifft2(psi_k)
    exp_factor = np.exp(-1j * V_finite * dt / (2 * hbar))
    psi = psi * exp_factor
    psi = np.where(np.isinf(V), 0, psi)
    return psi

# Evolve and store snapshots
psi_t = [psi.copy()]
for t in range(T_steps):
    psi = evolve_psi(psi, k2, dt, V)
    if t % 2 == 0:  # Store every 2nd frame for speed
        psi_t.append(psi.copy())

# Animation setup
fig, ax = plt.subplots(figsize=(7,6))
im = ax.imshow(np.abs(psi_t[0])**2, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', vmin=0, vmax=np.max(np.abs(psi_t[0])**2))
ax.set_title('Quantum Wave Packet Evolution')
ax.set_xlabel('x')
ax.set_ylabel('y')

def animate_q(i):
    im.set_data(np.abs(psi_t[i])**2)
    ax.set_title(f'Quantum Wave Packet Evolution (t={i*dt*2:.3f})')
    return [im]

ani_q = animation.FuncAnimation(fig, animate_q, frames=len(psi_t), blit=True, interval=50)

# Save as GIF
ani_q.save('stadium_quantum_wavepacket.gif', writer='pillow', fps=20)
plt.close(fig)
print("Quantum wave packet evolution GIF saved as 'stadium_quantum_wavepacket.gif'.")

Quantum wave packet evolution GIF saved as 'stadium_quantum_wavepacket.gif'.
